## ACS Housing Units by Building Size for Urban Villages, City of Phoenix

- https://www.census.gov/data/developers/data-sets/acs-5year.html
- https://api.census.gov/data/2013/acs/acs5/variables.html
- https://api.census.gov/data/2021/acs/acs5/variables.html

In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
import getters as get
import utilcalcs as calc
import geo_agg
from hou_size_vars import *

In [3]:
#Search parameters - NOTE DIFFERENT VARIABLE # ASSIGNMENTS IN 2010 VS 2020
y1 = '2021'
y0_5 = '2013'
y0_1 = '2010'

cols = 'group(B25024)'

source_1 = 'acs/acs1'
source_5 = 'acs/acs5'

#### Calculate building size columns function

In [4]:
# function to sum select building columns & recalc MOES
def group_bsize(df,year):
    df['u_1_E'] = df.loc[:,U1E].sum(axis=1)
    df['u_1_M'] = df.apply(lambda x: (calc.get_moe(x[U1M])),axis=1)
    df['u_24_E'] = df.loc[:,U24E].sum(axis=1)
    df['u_24_M'] = df.apply(lambda x: (calc.get_moe(x[U24M])),axis=1)
    df['u_520_E'] = df.loc[:,U520E].sum(axis=1)
    df['u_520_M'] = df.apply(lambda x: (calc.get_moe(x[U520M])),axis=1)
    df['u_o20_E'] = df.loc[:,Uo20E].sum(axis=1)
    df['u_o20_M'] = df.apply(lambda x: (calc.get_moe(x[Uo20M])),axis=1)
    df['u_oth_E'] = df.loc[:,UOthE].sum(axis=1)
    df['u_oth_M'] = df.apply(lambda x: (calc.get_moe(x[UOthM])),axis=1)
    df = df.rename(hou_rename,axis=1).drop(columns=hou_vars)
    return df

#### make urban villages from Census Block Groups
def make_uvil(geo_df,year): 
    df = get.get_bgp(source,year,cols)
    df = get.clean_data(df)
    df = group_bsize(df,year)
    
    #df = pd.merge(geo_df,df,how='left',left_on='geoid',right_on='GEO_ID')
    #df = df.drop(['geoid','GEO_ID'],axis=1)
    #df = geo_agg.sumgeo_cv(df,'name')
    return df

In [5]:
# make city and larger geos
def make_table(source,year,cols,geotype='big'):
    if geotype=='big':
        ph = get.get_phx(source,year,cols)
        mar = get.get_maricopa(source,year,cols)
        us = get.get_us(source,year,cols)
        az = get.get_az(source,year,cols)
        cit = get.get_comp_cities(source,year,cols)
        df = pd.concat([ph,mar,us,az,cit])
        df = df.drop(['NAME','us','state'],axis=1)
        aggtype='GEO_ID'
    elif geotype=='uv':
        df = get.get_bgp(source,year,cols)
        aggtype='name'
    else:
        pass
    df = get.clean_table(df)
    df = group_bsize(df,year)
    if geotype=='uv': df = make_uv(df,year)
    df = geo_agg.sumgeo_cv(df,aggtype)
    
    for col in df.columns[1:]:
        df.rename(columns={col:f'{col[:-1]}{year[-2:]}{col[-1:]}'},inplace=True)
    return df

## Calculate for Phoenix, Maricopa, larger geos

### Make a table with urban villages, city, and  U.S.

In [6]:
geo_y1_1y = make_table(source_1,y1,cols)
geo_y0_1y = make_table(source_1,y0_1,cols)

In [8]:
geo_y1_1y.columns

Index(['GEO_ID', 'u_520_21E', 'u_520_21M', 'u_520_21C', 'u_59_21E', 'u_59_21M',
       'u_59_21C', 'u_1019_21E', 'u_1019_21M', 'u_1019_21C', 'u_24_21E',
       'u_24_21M', 'u_24_21C', 'u_o50_21E', 'u_o50_21M', 'u_o50_21C',
       'u_oth_21E', 'u_oth_21M', 'u_oth_21C', 'u_1_21E', 'u_1_21M', 'u_1_21C',
       'u_2049_21E', 'u_2049_21M', 'u_2049_21C', 'u_tot_21E', 'u_tot_21M',
       'u_tot_21C', 'u_o20_21E', 'u_o20_21M', 'u_o20_21C'],
      dtype='object')

In [ ]:
change_vars = ['tot','1','24','59','1019','2049','o50','oth','520','o20']

In [ ]:
dfY1 = make_uvil(bgp_20,y1)

In [ ]:
dfY1.head(3)

In [ ]:
phY1 = make_table(y1,'phoenix')
phY1.rename(columns={'GEO_ID':'name'},inplace=True)

In [ ]:
phY1

In [ ]:
usY1 = make_table(y1,'us')
usY1.rename(columns={'GEO_ID':'name'},inplace=True)

In [ ]:
usY1

## Build current year table for all geos & export to Excel

In [ ]:
final = pd.concat([dfY1,phY1,usY1])
final.to_excel(f'output/Hou_bsize.xlsx',index=False)